In [1]:
import pydicom
import csv
import os

In [2]:
tcia_lgg_raw = '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/'
tcia_gbm_raw = '/anvar/public_datasets/tcia/manifest-21CCx8fx964169794518489879/TCGA-GBM/'
len(os.listdir(tcia_lgg_raw)),len(os.listdir(tcia_gbm_raw))

(67, 136)

In [3]:
import numpy as np

sub_ids = next(os.walk(tcia_lgg_raw))[1]

## First, find FLAIR, T2, and T1-mprage scans based on expected file names
flair_paths=[]
t2_paths=[]
t1_paths=[]

for (root, __, __) in os.walk(tcia_lgg_raw):
    ##   FLAIR
    if  root.find('FLAIR') >0 and\
        root.find('T1') <0:
        flair_paths.append(root)
    if  root.find('flair') >0 and\
        root.find('T1') <0:
        flair_paths.append(root)
    if  root.find('Flair') >0 and\
        root.find('T1') <0:
        flair_paths.append(root)
    if  root.find('3.000000-58602') >0:
        flair_paths.append(root)
        

    ##   T2
    if  root.find('T2') > 0 and\
        root.find('FLAIR')<0:
        t2_paths.append(root)
    if  root.find('t2') > 0 and\
        root.find('FLAIR')< 0:
        t2_paths.append(root)
    if  root.find('FSE') > 0 and\
        root.find('T1')< 0:
        t2_paths.append(root)   
    if  root.find('fse') > 0 and\
        root.find('T1')< 0:
        t2_paths.append(root)   
    if  root.find('DOUBLEECHO') > 0 and\
        root.find('FLAIR')< 0:
        t2_paths.append(root)
        
    ##   T1 pre
    if  root.find('T1') > 0 or\
        root.find('t1') > 0 or\
        root.find('MP RAGE') > 0 or\
        root.find('VOLUMAX') >0 or\
        root.find('VOLUMETRIC') >0 or\
        root.find('SPGR') > 0:
        t1_paths.append(root)

        
flair_paths = np.unique(flair_paths).tolist()
t2_paths = np.unique(t2_paths).tolist()
t1_paths = np.unique(t1_paths).tolist()

In [4]:
print(len(sub_ids), len(flair_paths), len(t2_paths), len(t1_paths))

66 69 90 312


In [5]:
# creating a list of subjects with package of four images
import pandas as pd
df_result = pd.DataFrame(columns = ['ID', 'T1', 'T1C', 'T2', 'FLAIR'])
num_tag = pydicom.tag.Tag(int('0020', 16), int('0011', 16)) # series number tag

for idx, sub in enumerate(sub_ids):
    # flair
    df_result.at[idx,'ID'] = sub
    fn_paths = list(filter(lambda x: sub in x, flair_paths)) # grab paths for this sub
    if len(fn_paths) == 1:
        df_result.at[idx,'FLAIR'] = fn_paths[0]
    # t2
    fn_paths = list(filter(lambda x: sub in x, t2_paths)) # grab paths for this sub
    if len(fn_paths) == 1:
        df_result.at[idx,'T2'] = fn_paths[0]

    # t1: comparing two with num tags
    fn_paths = list(filter(lambda x: sub in x, t1_paths)) # grab paths for this sub
    num_tag_list = []
    for fn_path in fn_paths:
        ds = pydicom.dcmread(fn_path+'/'+os.listdir(fn_path)[0])
        num_tag_list.append(ds[num_tag].value)
    try:
        df_result.at[idx,'T1C'] = fn_paths[num_tag_list.index(max(num_tag_list))]
        df_result.at[idx,'T1'] = fn_paths[num_tag_list.index(min(num_tag_list))]
    except:
        pass

In [6]:
df_result = pd.DataFrame(columns = ['ID', 'T1', 'T1C', 'T2', 'FLAIR'])
num_tag = pydicom.tag.Tag(int('0020', 16), int('0011', 16)) # series number tag

for idx, sub in enumerate(sub_ids):
    # flair
    df_result.at[idx,'ID'] = sub
    fn_paths = list(filter(lambda x: sub in x, flair_paths)) # grab paths for this sub
    if len(fn_paths) > 0:
        df_result.at[idx,'FLAIR'] = fn_paths[0]
    # t2
    fn_paths = list(filter(lambda x: sub in x, t2_paths)) # grab paths for this sub
    if len(fn_paths) > 0:
        df_result.at[idx,'T2'] = fn_paths[0]

    # t1: comparing two with num tags
    fn_paths = list(filter(lambda x: sub in x, t1_paths)) # grab paths for this sub
    num_tag_list = []
    for fn_path in fn_paths:
        ds = pydicom.dcmread(fn_path+'/'+os.listdir(fn_path)[0])
        num_tag_list.append(ds[num_tag].value)
    try:
        df_result.at[idx,'T1C'] = fn_paths[num_tag_list.index(max(num_tag_list))]
        df_result.at[idx,'T1'] = fn_paths[num_tag_list.index(min(num_tag_list))]
    except:
        pass

In [8]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 0 to 65
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      66 non-null     object
 1   T1      65 non-null     object
 2   T1C     65 non-null     object
 3   T2      64 non-null     object
 4   FLAIR   61 non-null     object
dtypes: object(5)
memory usage: 5.1+ KB


In [9]:
df_result[df_result.isna().any(axis=1)]

,ID,T1,T1C,T2,FLAIR
6,TCGA-CS-4942,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,NaN,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...
10,TCGA-DU-6402,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,NaN,NaN
40,TCGA-CS-4938,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,NaN
45,TCGA-CS-5390,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,NaN
54,TCGA-CS-5394,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,NaN
60,TCGA-DU-8158,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,NaN
65,TCGA-DU-6395,NaN,NaN,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...,/anvar/public_datasets/tcia/manifest-25ZdsnMy5...


In [10]:
df_result[df_result['ID'] == 'TCGA-DU-6395'].values

array([['TCGA-DU-6395', nan, nan,
        '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-6395/02-09-1983-BRAINGD-38915/2.000000-ScanningSequence fse-66259',
        '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-6395/02-09-1983-BRAINGD-38915/3.000000-ScanningSequence flair-95070']],
      dtype=object)

In [12]:
df_result.at[12,'T2'] = '/home/datasets/@Uranus/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-5393/06-06-1999-JHN MRI BRAINSTRYKER IP-28540/5.000000-BRAINCONTRASTDOUBLEECHO-62620/'
df_result.at[14,'FLAIR'] = '/home/datasets/@Uranus/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-6402/12-31-1983-WAND BRAIN-73244/1.000000-axial diff.-30337'
df_result.at[14, 'T2'] = '/home/datasets/@Uranus/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-6402/12-31-1983-WAND BRAIN-73244/4.000000-axial perf.-20953'
df_result.at[18, 'FLAIR'] = '/home/datasets/@Uranus/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-4938/04-15-1996-JHN BRAIN MR OP-84760/2.000000-FASTBRAINEPIDIFF-74032'
df_result.at[19, 'FLAIR'] ='/home/datasets/@Megatron/druzhinina/tcia/manifest-21CCx8fx964169794518489879/TCGA-LGG/TCGA-CS-5390/03-07-1996-BRAINSTRYKER-97078/5.000000-BRAINSTRYKERDIFFUSION-72204'
df_result.at[40, 'T2'] ='/home/datasets/@Uranus/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-4942/TCGA-CS-4942/02-22-1997-MRI BRAIN WITHO-04689/401.000000-DUAL AX SE-07653'
df_result.at[45, 'FLAIR'] = '/home/datasets/@Uranus/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-5394/12-05-1995-JHN BRAIN-97935/4.000000-RETROORBITT2LATEECHOB-49308'
df_result.at[55, 'FLAIR'] = '/home/datasets/@Uranus/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-8158/09-22-1986-BRAIN WWO-21326/4.000000-DIFF 1000 24 FOV-88158'
df_result.at[65, 'T1'] = '/home/datasets/@Uranus/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-6395/02-09-1983-BRAINGD-38915/4.000000-ScanningSequence memp-03903'
df_result.at[65, 'T1C'] = '/home/datasets/@Uranus/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-6395/02-09-1983-BRAINGD-38915/5.000000-ScanningSequence memp-22371'

In [79]:
# df_result.to_csv('lgg_paths.csv')

In [7]:
paths = pd.read_csv('lgg_paths.csv')

## Saving folders

In [17]:
#!conda install -c conda-forge dcm2niix

In [42]:
import glob2
len(glob2.glob(savedir +'/**/t1.nii.gz'))

107

In [20]:
paths = pd.read_csv('lgg_paths.csv')

In [88]:
os.listdir('/anvar/public_datasets/tcia/training_data')

['TCGA-02-0086',
 'TCGA-02-0048',
 'TCGA-02-0102',
 'TCGA-02-0059',
 'TCGA-02-0069',
 'TCGA-19-2624',
 'TCGA-06-0182',
 'TCGA-08-0360',
 'TCGA-06-0168',
 'TCGA-02-0006',
 'TCGA-19-2631',
 'TCGA-06-0133',
 'TCGA-12-1094',
 'TCGA-12-1602',
 'TCGA-06-2570',
 'TCGA-06-0119',
 'TCGA-06-0145',
 'TCGA-08-0509',
 'TCGA-06-0649',
 'TCGA-06-5408',
 'TCGA-02-0116',
 'TCGA-06-0188',
 'TCGA-76-6661',
 'TCGA-02-0047',
 'TCGA-12-3650',
 'TCGA-08-0354',
 'TCGA-02-0064',
 'TCGA-08-0529',
 'TCGA-14-1825',
 'TCGA-06-0162',
 'TCGA-06-0213',
 'TCGA-02-0033',
 'TCGA-08-0359',
 'TCGA-06-0241',
 'TCGA-08-0357',
 'TCGA-02-0054',
 'TCGA-06-0184',
 'TCGA-08-0358',
 'TCGA-12-1601',
 'TCGA-06-5413',
 'TCGA-14-1829',
 'TCGA-76-6191',
 'TCGA-02-0060',
 'TCGA-02-0075',
 'TCGA-06-0177',
 'TCGA-06-5412',
 'TCGA-06-0154',
 'TCGA-08-0350',
 'TCGA-08-0521',
 'TCGA-06-0179',
 'TCGA-19-5951',
 'TCGA-12-1598',
 'TCGA-14-0789',
 'TCGA-12-0616',
 'TCGA-02-0070',
 'TCGA-06-1084',
 'TCGA-12-0829',
 'TCGA-76-4932',
 'TCGA-02-0068

In [31]:
basedir = '/anvar/public_datasets/tcia/'
savedir = '/anvar/public_datasets/preproc_study/lgg/orig/'

In [25]:
os.makedirs('/anvar/public_datasets/preproc_study/lgg/orig/')

In [33]:
import dicom2nifti
import dicom2nifti.settings as settings

# dicom2nifti.settings.disable_validate_slicecount()

for i in range(0, len(paths)):
    print(paths['ID'].iloc[i])
    try:
        os.mkdir(savedir + paths['ID'].iloc[i])
    except OSError as error:
#         print(error) 
        pass
    
    try:
        dicom2nifti.dicom_series_to_nifti(basedir + paths['FLAIR'].iloc[i][1:], 
                                          savedir + paths['ID'].iloc[i] + '/flair.nii.gz')
        dicom2nifti.dicom_series_to_nifti(basedir + paths['T2'].iloc[i][1:], 
                                          savedir + paths['ID'].iloc[i] + '/t2.nii.gz')
        dicom2nifti.dicom_series_to_nifti(basedir + paths['T1C'].iloc[i][1:], 
                                          savedir + paths['ID'].iloc[i] + '/t1c.nii.gz')
        dicom2nifti.dicom_series_to_nifti(basedir + paths['T1'].iloc[i][1:], 
                                          savedir + paths['ID'].iloc[i] + '/t1.nii.gz')
    except Exception as e:
        print(e)
        print('problems', paths['ID'].iloc[i]) 
        pass

TCGA-DU-7019
TCGA-DU-7302
TCGA-DU-5852
TCGA-DU-7014


Slice increment not consistent through all slices
---------------------------------------------------------
[-120.  105. -110.] [0. 0. 0.]
[-120.  105. -104.] [ 0.  0. -6.]
Instance Number: 6
---------------------------------------------------------


SLICE_INCREMENT_INCONSISTENT
problems TCGA-DU-7014
TCGA-CS-6669
TCGA-DU-6405


Slice increment not consistent through all slices
---------------------------------------------------------
[-120.  117.  -60.] [0. 0. 0.]
[-120.  117.  -57.] [ 0.  0. -3.]
Instance Number: 3
---------------------------------------------------------


SLICE_INCREMENT_INCONSISTENT
problems TCGA-DU-6405
TCGA-DU-5851


...

SLICE_INCREMENT_INCONSISTENT
problems TCGA-DU-6397
TCGA-DU-6399


Slice increment not consistent through all slices
---------------------------------------------------------
[-120.  120.  -62.] [0. 0. 0.]
[-120.  120.  -59.] [ 0.  0. -3.]
Instance Number: 3
---------------------------------------------------------


SLICE_INCREMENT_INCONSISTENT
problems TCGA-DU-6399
TCGA-DU-7018
TCGA-DU-8168
TCGA-DU-6395


Slice increment not consistent through all slices
---------------------------------------------------------
[-120.        120.        -72.300003] [0. 0. 0.]
[-120.        120.        -69.300003] [ 0.  0. -3.]
Instance Number: 003
---------------------------------------------------------


SLICE_INCREMENT_INCONSISTENT
problems TCGA-DU-6395


In [11]:
os.listdir('/anvar/public_datasets/preproc_study/lgg/orig/TCGA-CS-6188')

['T2.nii.gz',
 'FLAIR.nii.gz',
 'CT1.json',
 'FLAIRa.json',
 'flair.nii.gz',
 'T2a.nii.gz',
 'T1.nii.gz',
 't1.nii.gz',
 'T2b.nii.gz',
 'FLAIRb.nii.gz',
 'T1a.nii.gz',
 'T2.json',
 'T1b.nii.gz',
 'CT1a.nii.gz',
 'T1.json',
 'FLAIRa.nii.gz',
 'T2c.nii.gz',
 'CT1.nii.gz',
 't2.nii.gz',
 'FLAIR.json',
 'T2a.json',
 'CT1a.json',
 't1c.nii.gz',
 'CT1b.nii.gz',
 'T1a.json']

## Save GBM datasets

In [89]:
paths = pd.read_csv('gbm_paths.csv')
basedir = '/anvar/public_datasets/tcia/'
savedir = '/anvar/public_datasets/preproc_study/gbm/orig/'
# os.makedirs('/anvar/public_datasets/preproc_study/gbm/orig/')

In [38]:
dicom2nifti.settings.disable_validate_slicecount()

for i in range(0, len(paths)):
    print(paths['ID'].iloc[i])
    try:
        os.mkdir(savedir + paths['ID'].iloc[i])
    except OSError as error:
#         print(error) 
        pass
    
    try:
      
        dicom2nifti.dicom_series_to_nifti(basedir + paths['T1C'].iloc[i][1:], 
                                          savedir + paths['ID'].iloc[i] + '/t1c.nii.gz')
        dicom2nifti.dicom_series_to_nifti(basedir + paths['T1'].iloc[i][1:], 
                                          savedir + paths['ID'].iloc[i] + '/t1.nii.gz')
        dicom2nifti.dicom_series_to_nifti(basedir + paths['FLAIR'].iloc[i][1:], 
                                          savedir + paths['ID'].iloc[i] + '/flair.nii.gz')
        dicom2nifti.dicom_series_to_nifti(basedir + paths['T2'].iloc[i][1:], 
                                          savedir + paths['ID'].iloc[i] + '/t2.nii.gz')
    except Exception as e:
        print(e)
        print('problems', paths['ID'].iloc[i]) 
        pass

TCGA-02-0034
TCGA-06-0158


Slice increment not consistent through all slices
---------------------------------------------------------
[-120.  123.  -80.] [0. 0. 0.]
[-120.  123.  -77.] [ 0.  0. -3.]
Instance Number: 3
---------------------------------------------------------


SLICE_INCREMENT_INCONSISTENT
problems TCGA-06-0158
TCGA-76-4934
TCGA-06-0187


Slice increment not consistent through all slices
---------------------------------------------------------
[-119.0121918 -120.5853653  -98.9268341] [0. 0. 0.]
[-119.0121918 -120.5853653  -93.9268341] [ 0.  0. -5.]
Instance Number: 4
---------------------------------------------------------


SLICE_INCREMENT_INCONSISTENT
problems TCGA-06-0187
TCGA-14-1829
TCGA-08-0356
TCGA-06-0137


...

TCGA-06-0154
SLICE_INCREMENT_INCONSISTENT
problems TCGA-06-0154
TCGA-02-0116
TCGA-02-0064
TCGA-12-1602
TCGA-19-5951
TCGA-76-6662
TCGA-12-1601
TCGA-06-0182
NOT_A_VOLUME
problems TCGA-06-0182
TCGA-02-0046
TCGA-08-0522
TCGA-06-0139
TCGA-27-1838
TCGA-06-0188


Slice increment not consistent through all slices
---------------------------------------------------------
[-126.0365906 -118.8292694  -77.2195129] [0. 0. 0.]
[-126.0365906 -118.8292694  -72.2195129] [ 0.  0. -5.]
Instance Number: 4
---------------------------------------------------------


SLICE_INCREMENT_INCONSISTENT
problems TCGA-06-0188
TCGA-06-0177


Slice increment not consistent through all slices
---------------------------------------------------------
[-119.531  -134.165   -86.6344] [0. 0. 0.]
[-119.531  -134.165   -81.6344] [ 0.  0. -5.]
Instance Number: 4
---------------------------------------------------------


SLICE_INCREMENT_INCONSISTENT
problems TCGA-06-0177
TCGA-27-2526
TCGA-02-0068


## Lets see the dimentions

In [ ]:
list_patients = paths['ID']
list_patients[0]

In [55]:
import numpy as np
import ants

sizes_frame = pd.DataFrame(columns = ['id','t1_x', 't1_y', 't1_z', 't1c_x', 't1c_y', 't1c_z'])

for i in list_patients:
    try:
        t1_size = np.shape(ants.image_read(savedir + i + '/t1.nii.gz').numpy())
        t1c_size = np.shape(ants.image_read(savedir + i + '/t1c.nii.gz').numpy())
        sizes_frame = sizes_frame.append(
        {'id': i,
        't1_x':t1_size[0],
        't1_y':t1_size[1],
        't1_z':t1_size[2],
        't1c_x':t1c_size[0],
        't1c_y':t1c_size[1],
        't1c_z':t1c_size[2]}, ignore_index=True
        )
    except Exception as e:
        print(e)
        pass

File /anvar/public_datasets/preproc_study/gbm/orig/TCGA-06-0158/t1.nii.gz does not exist!
File /anvar/public_datasets/preproc_study/gbm/orig/TCGA-06-0187/t1.nii.gz does not exist!
File /anvar/public_datasets/preproc_study/gbm/orig/TCGA-06-0137/t1.nii.gz does not exist!
/build100/ANTsPy/itksource/Modules/IO/NIFTI/src/itkNiftiImageIO.cxx:1972:
itk::ERROR: ITK only supports orthonormal direction cosines.  No orthonormal definition found!
File /anvar/public_datasets/preproc_study/gbm/orig/TCGA-02-0075/t1.nii.gz does not exist!
File /anvar/public_datasets/preproc_study/gbm/orig/TCGA-06-0175/t1.nii.gz does not exist!
File /anvar/public_datasets/preproc_study/gbm/orig/TCGA-06-0240/t1.nii.gz does not exist!
File /anvar/public_datasets/preproc_study/gbm/orig/TCGA-06-0162/t1.nii.gz does not exist!
File /anvar/public_datasets/preproc_study/gbm/orig/TCGA-06-0184/t1.nii.gz does not exist!
/build100/ANTsPy/itksource/Modules/IO/NIFTI/src/itkNiftiImageIO.cxx:1972:
itk::ERROR: ITK only supports orthono

In [54]:
os.listdir('/anvar/public_datasets/preproc_study/gbm/orig/TCGA-06-0168/')

['flair.nii.gz']

### Collecting paths

In [56]:
paths = pd.read_csv('gbm_paths.csv')
basedir = '/anvar/public_datasets/tcia/'
savedir = '/anvar/public_datasets/preproc_study/gbm/dicom/'
os.makedirs('/anvar/public_datasets/preproc_study/gbm/dicom/')

In [58]:
import shutil

for i in range(0, len(paths)):
    print(paths['ID'].iloc[i])
    try:
#         os.mkdir(basedir + paths['ID'].iloc[i])
        shutil.copytree(basedir + paths['T1'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/T1/') 
        shutil.copytree(basedir + paths['T1C'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/CT1/') 
        shutil.copytree(basedir + paths['T2'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/T2') 
        shutil.copytree(basedir + paths['FLAIR'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/FLAIR/')
        
    except OSError as error:
        print(error) 
        pass  

TCGA-02-0034
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-02-0034'
TCGA-06-0158
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-06-0158'
TCGA-76-4934
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-76-4934'
TCGA-06-0187
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-06-0187'
TCGA-14-1829
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-14-1829'
TCGA-08-0356
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-08-0356'
TCGA-06-0137
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-06-0137'
TCGA-06-0649
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-06-0649'
TCGA-06-0646
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-06-0646'
TCGA-02-0069
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-02-0069'
TCGA-06-0241
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-06-0241'
TCGA-06-0192
[Errno 17] File exists: '/anvar/public_datasets/tcia/CGA-06-0192'
TCGA-12-1094
[Errno 17] File exists: '/anvar/public_

In [102]:
paths = pd.read_csv('lgg_paths.csv')
basedir = '/anvar/public_datasets/tcia/'
savedir = '/anvar/public_datasets/preproc_study/lgg/dicom/'
# os.makedirs('/anvar/public_datasets/preproc_study/lgg/dicom/')

In [103]:
paths['FLAIR'][paths['ID'] == 'TCGA-CS-5390'] = ' manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-5390/03-07-1996-BRAINSTRYKER-97078/5.000000-BRAINSTRYKERDIFFUSION-72204'

In [104]:
paths['T2'][paths['ID'] == 'TCGA-CS-4942'] = ' manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-4942/02-22-1997-MRI BRAIN WITHO-04689/401.000000-DUAL AX SE-07653'

In [106]:
# paths.to_csv('lgg_paths.csv')

In [15]:
os.listdir('/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-7015/')

['T1', 'T2', 'CT1', 'FLAIR']

In [105]:
import shutil

for i in range(0, len(paths)):
    print(paths['ID'].iloc[i])
    try:
        os.mkdir(basedir + paths['ID'].iloc[i])
        shutil.copytree(basedir + paths['T1'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/T1/') 
        shutil.copytree(basedir + paths['T1C'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/CT1/') 
        shutil.copytree(basedir + paths['T2'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/T2') 
        shutil.copytree(basedir + paths['FLAIR'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/FLAIR/')
        
    except OSError as error:
        print(error) 
        pass  

TCGA-DU-7019
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-7019/T2'
TCGA-DU-7302
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-7302/T2'
TCGA-DU-5852
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-5852/T2'
TCGA-DU-7014
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-7014/T2'
TCGA-CS-6669
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6669/T2'
TCGA-DU-6405
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-6405/T2'
TCGA-DU-5851
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-5851/T2'
TCGA-DU-5854
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-5854/T2'
TCGA-DU-6408
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-6408/T2'
TCGA-DU-5874
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/lgg/dic

In [80]:
import glob
glob.glob('/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-GBM/TCGA-02-0086/*/*')

rtdir = '/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-GBM/'

In [81]:
len(os.listdir(rtdir))

103

In [84]:
len(glob.glob(savedir + '/*/' + '/RTSTRUCT/'))

102

In [75]:
import shutil

for i in range(0, len(paths)):
    print(paths['ID'].iloc[i])
    try:
#         os.mkdir(savedir + paths['ID'].iloc[i] + '/')
        print(glob.glob(rtdir + paths['ID'].iloc[i] +'/*/'))
        shutil.copytree(glob.glob(rtdir + paths['ID'].iloc[i] +'/*/')[0], savedir + paths['ID'].iloc[i] + '/RTSTRUCT/') 
      
    except Exception as error:
        print(error) 
        pass  
          

TCGA-02-0034
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/']
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/gbm/dicom/TCGA-02-0034/RTSTRUCT/'
TCGA-06-0158
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-GBM/TCGA-06-0158/09-05-1996-30968/']
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/gbm/dicom/TCGA-06-0158/RTSTRUCT/'
TCGA-76-4934
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-GBM/TCGA-76-4934/10-08-2000-MRI BRAINSTRYKER IP-29203/']
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/gbm/dicom/TCGA-76-4934/RTSTRUCT/'
TCGA-06-0187
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-GBM/TCGA-06-0187/07-07-2004-41002/']
[Errno 17] File exists: '/anvar/public_datasets/preproc_study/gbm/dicom/TCGA-06-0187/RTSTRUCT/'
TCGA-14-1829
[]
list index out of range
TCGA-08-0356
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-GBM/TCGA-08-0356/03-26-2002-MS